## PROYECTO CURSO PYSPARK

Autor: Martin Fierro

## Descripción General del Proyecto



### 1. Selección de la Fuente de Datos

- **Nombre del Dataset**: NYC Taxi Trips (Yellow, Green o FHV Trips).  
- **Proveedores**:  
  - [NYC Taxi & Limousine Commission (TLC)](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page), que publica mensualmente todos los viajes realizados.  
  - Copias alternativas en [Kaggle](https://www.kaggle.com/datasets), que a veces tienen los archivos consolidados en diferentes formatos.  
- **Volumen**: Se puede alcanzar fácilmente cientos de millones de registros si se descargan varios años de datos. Esto justifica el uso de técnicas de optimización y escalabilidad.

### 2. Objetivos de Negocio y Analíticos

1. **Análisis de Demanda de Taxis**:  
   - Identificar los horarios de mayor afluencia en distintas zonas de la ciudad.  
   - Calcular el volumen de viajes por día/semana/mes.  

2. **Análisis de Duración y Distancia de Viajes**:  
   - Estimar el tiempo promedio de recorrido entre distintas zonas, o el tiempo total de cada jornada.  
   - Detectar picos de duración (por congestión, eventos masivos, etc.).

3. **Costos y Propinas**:  
   - Relacionar montos de propinas con la hora del día, el día de la semana o la zona de recogida.  
   - Evaluar patrones de pagos en efectivo vs. tarjeta.

4. **Optimización de Consultas y Procesamiento**:  
   - Aplicar las técnicas de particionamiento (por fecha, zona o ambas).  
   - Usar Z-Ordering para columnas clave, como fecha/hora o zonas geográficas.  
   - Aplicar broadcast joins y manejo de skew con tablas auxiliares (por ejemplo, dimensiones de zonas o festivos).  
   - Caching y persistencia para exploraciones repetitivas.

## Librerias

In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, min, max, dayofmonth, month, year, hour

## 1. Adquisición y Preparación de Datos

1. **Descarga de Datos**  
   - Obtener los archivos CSV o Parquet de uno o varios años de registros de taxi (p. ej. Yellow Taxi Trip Records).  
   - Verificar la disponibilidad de los campos (por ejemplo: pickup_datetime, dropoff_datetime, pickup_location, dropoff_location, passenger_count, trip_distance, total_amount, etc.).

2. **Limpieza y Estandarización**  
   - Eliminar filas con datos claramente inconsistentes (distancias negativas, tiempos de viaje nulos, etc.).  
   - Convertir tipos de columnas (fecha/hora, numéricos, etc.) y unificar formatos.  
   - Agregar columnas derivadas si se requiere (por ejemplo, “hora del día” de la recogida, “día de la semana”, “zona geográfica” unificada a partir de lat/long o “LocationID” si se usa la tabla de zonas).

3. **Enriquecimiento con Tablas Auxiliares**  
   - Incorporar la tabla “Taxi Zone Lookup” que relaciona cada LocationID con un barrio o zona (por ejemplo, Bronx, Brooklyn, Manhattan, etc.).  
   - (Opcional) Añadir información de clima o de eventos de la ciudad para un análisis predictivo más sofisticado.

In [4]:
# Crear una SparkSession
spark = SparkSession.builder \
    .appName("Leer archivo Parquet") \
    .getOrCreate()

In [55]:
LRF_codigos = spark.read.csv("taxi_zone_lookup.csv", header=True, inferSchema=True)

In [56]:
LRF_codigos = LRF_codigos.select("LocationID","Borough")

In [6]:
# Leer el archivo Parquet
enero = spark.read.parquet("enero.parquet")

# Opcional: Mostrar el esquema del DataFrame
enero.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [10]:
numero_filas = enero.count()
numero_filas

2964624

In [8]:
nulos_por_columna = enero.select(
    [
        sum(col(c).isNull().cast("int")).alias(c) for c in enero.columns
    ]
)

nulos_por_columna.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       0|                   0|                    0|         140162|            0|    140162|            140162|           0|           0|           0|          0|    0|      0|         

In [60]:
clean_df = enero.filter(col("trip_distance") > 0)

In [61]:
clean_df.count()

2904253

In [62]:
clean_df= clean_df.withColumn("dia_pickup", dayofmonth(col("tpep_pickup_datetime"))) \
    .withColumn("mes_pickup", month(col("tpep_pickup_datetime"))) \
    .withColumn("hora_pickup", hour(col("tpep_pickup_datetime")))

In [63]:
clean_df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+----------+----------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|dia_pickup|mes_pickup|hora_pickup|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+----------+----------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|  

In [64]:
clean_df = clean_df.join(LRF_codigos, clean_df["PULocationID"] == LRF_codigos["LocationID"], how="left")

## 2. Particionamiento y Clustering

1. **Diseño del Esquema de Particionamiento**  
   - Decidir si se particionará por **año/mes** de la fecha de recogida (pickup_datetime) o por zona geográfica, o incluso una combinación de ambas (aunque se debe evitar crear un exceso de particiones).  
   - Justificar la decisión en función de las consultas más frecuentes (por ejemplo, filtros por fecha o análisis por zonas).

2. **Implementación y Medición de Rendimiento**  
   - Crear un directorio (o tabla en Delta, si se usa un lago de datos) particionado por la columna elegida.  
   - Medir el tamaño total de los archivos y el número de archivos resultantes.  
   - Ejecutar algunas consultas comparativas antes y después del particionado, anotando el tiempo requerido.

3. **Liquid Clustering / Auto Optimize** (según la plataforma)  
   - Describir cómo se implementarían técnicas de reescritura dinámica para mantener los datos optimizados a medida que se realizan inserciones o actualizaciones (en caso de que se use un formato como Delta Lake).  
   - Evaluar la variación en el tamaño promedio de los archivos y la latencia de las consultas tras esta optimización.

In [36]:
clean_df.groupBy("mes_pickup").count().show()

+----------+-------+
|mes_pickup|  count|
+----------+-------+
|        12|     12|
|         1|2904238|
|         2|      3|
+----------+-------+



In [37]:
clean_df.groupBy("PULocationID").count().show()

+------------+-----+
|PULocationID|count|
+------------+-----+
|         148|27111|
|         243|  409|
|          31|    5|
|         137|32215|
|         251|    2|
|          85|  174|
|          65| 1310|
|         255|  964|
|          53|   38|
|         133|  110|
|          78|  178|
|         108|  110|
|         155|  195|
|         211|19793|
|          34|   55|
|         193|  582|
|         126|   84|
|         101|   41|
|          81|  113|
|          28|  394|
+------------+-----+
only showing top 20 rows



In [66]:
clean_df.groupBy("Borough").count().show()

+-------------+-------+
|      Borough|  count|
+-------------+-------+
|       Queens| 263020|
|          EWR|     53|
|      Unknown|   9683|
|     Brooklyn|  22550|
|Staten Island|     50|
|          N/A|    786|
|    Manhattan|2602319|
|        Bronx|   5792|
+-------------+-------+



In [51]:
test = clean_df.filter(col("Borough") == "N/A")

In [50]:
test.count()

2894570